# GRU: Generate obama speeches

Using truncated back propagation and add embedding layer instead of one-hot encoding going into GRU.

In [1]:
import pandas as pd
import numpy as np
import math
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
#from torch.nn.functional import softmax
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
np.set_printoptions(precision=2, suppress=True, linewidth=3000, threshold=20000)
from typing import Sequence

dtype = torch.float
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [2]:
import codecs
def get_text(filename:str):
    """
    Load and return the text of a text file, assuming latin-1 encoding as that
    is what the BBC corpus uses.  Use codecs.open() function not open().
    """
    with codecs.open(filename, mode='r') as f:
        s = f.read()
    return s

In [3]:
def normal_transform(x, mean=0.0, std=0.01):
    "Convert x to have mean and std"
    return x*std + mean

def randn(n1, n2,          
          mean=0.0, std=0.01, requires_grad=False,
          device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),
          dtype=torch.float64):
    x = torch.randn(n1, n2, device=device, dtype=dtype)
    x = normal_transform(x, mean=mean, std=std)
    x.requires_grad=requires_grad
    return x

In [4]:
def plot_history(history, yrange=(0.0, 5.00), figsize=(3.5,3)):
    plt.figure(figsize=figsize)
    plt.ylabel("Sentiment log loss")
    plt.xlabel("Epochs")
    loss = history[:,0]
    valid_loss = history[:,1]
    plt.plot(loss, label='train_loss')
    plt.plot(valid_loss, label='val_loss')
    # plt.xlim(0, 200)
    plt.ylim(*yrange)
    plt.legend()#loc='lower right')
    plt.show()

In [5]:
def getvocab(strings):
    letters = [list(l) for l in strings]
    vocab = set([c for cl in letters for c in cl])
    vocab = sorted(list(vocab))
    ctoi = {c:i for i, c in enumerate(vocab)}
    return vocab, ctoi

In [6]:
def softmax(y):
    expy = torch.exp(y)
    if len(y.shape)==1: # 1D case can't use axis arg
        return expy / torch.sum(expy)
    return expy / torch.sum(expy, axis=1).reshape(-1,1)

def cross_entropy(y_prob, y_true):
    """
    y_pred is n x k for n samples and k output classes and y_true is n x 1
    and is often softmax of final layer.
    y_pred values must be probability that output is a specific class.
    Binary case: When we have y_pred close to 1 and y_true is 1,
    loss is -1*log(1)==0. If y_pred close to 0 and y_true is 1, loss is
    -1*log(small value) = big value.
    y_true values must be positive integers in [0,k-1].
    """
    if torch.isnan(y_prob).any():
        raise ValueError("cross_entropy: y_prob has NaN!",y_prob)
    n = y_prob.shape[0]
    # Get value at y_true[j] for each sample with fancy indexing
    p = y_prob[range(n),y_true]
    p_ = p.detach()
    if torch.isnan(p).any():
        raise ValueError("cross_entropy: p has NaN! p=",p_,"y_prob=",y_prob)
    if (p_<0).any():
        raise ValueError("cross_entropy: y_prob has negative value!:",p_)
    m = torch.mean(-torch.log(p))
    if torch.isnan(m):
        raise ValueError("cross_entropy: mean is NaN! p=",p_)
    return m

## Load and split into chunks

The stochastic part of SGD is critical for training models. The idea is simply to use a small subset of the data when computing gradients to update the model parameters. Generally we take a small batch size of say 32 records, run that through the model, and then compute a loss. From that loss we compute the gradient and then update the model parameters and move onto the next batch.  Once all batches are complete, we have completed an epoch.  We should shuffle the batches and keep going.

We can also be stochastic by updating the gradient in the middle of long sequences, rather than waiting until after a complete batch of long sequences.  If the sequences are really long, waiting till the end of a batch reduces the stochastic nature. Instead I'm going to try breaking up the entire input into a small number of very long sequences. In this way the RNN can keep the hidden state going for the complete sequence. Of course the only problem is that we cannot compute back propagation that far, so at some sequence length I can update the gradient and wipe it out then continue. I think this is easier than modifying the data set stride so that a standard training loop for an RNN keeps the same hidden state across long sequences even if we have broken into chunks.

Let's say that we have a large text and we break it up into six chunks: A,B,C,D,E,F. then, six is our batch size and we will process each long sequence exactly once per epic. However to get stochastic nature, we will update the gradient after only a small sequence of characters.  We pick the chunk size and then the batch sizes computed instead of having to specify both. I think the chunk size is more important: how much can you store in a single hidden state vector.

Come to think of it, all we need to specify is the number of chunks we want to break the text into.  There won't be any batch size because we have a single batch with `nchunks`  long records in it.

In [7]:
text = get_text("data/obama-speeches.txt").lower() # generated from obama-sentences.py
len(text)

4224143

In [8]:
text = text[0:1_000_000] # testing
n = len(text)

bptt = 100                # only look back this many time steps for gradients
nhidden = 1024
char_embed_sz = 60        # there are 50+ chars, squeeze down into fewer dimensions for embedding prior to input into RNN 
nchunks = 64              # break up the input into a number of chunks (doesn't have to be small like batch size)
chunk_size = n // nchunks # the sequences will be very long
n = nchunks * chunk_size  # reset size so it's an even multiple of chunk size
text = text[0:n]

In [9]:
vocab, ctoi = getvocab(text)

In [10]:
chunks = [text[p:p+chunk_size] for p in range(0, n, chunk_size)]
X = torch.empty(nchunks, chunk_size-1, device=device, dtype=torch.long) # int8 doesn't work as indices
y = torch.empty(nchunks, chunk_size-1, device=device, dtype=torch.long)
for i,chunk in enumerate(chunks):
    X[i,:] = torch.tensor([ctoi[c] for c in chunk[0:-1]], device=device)
    y[i,:] = torch.tensor([ctoi[c] for c in chunk[1:]],   device=device)
    
# X, y are now chunked and numericalized into big 2D matrices

In [11]:
nclasses = len(ctoi)
print(f"{nchunks:,d} training records, chunk length {chunk_size}, vocab size {len(ctoi)}, char_embed_sz {char_embed_sz}, state is {nhidden}-vector")

64 training records, chunk length 15625, vocab size 70, char_embed_sz 60, state is 1024-vector


In [12]:
X.shape, nchunks

(torch.Size([64, 15624]), 64)

In [13]:
X[:,0].shape

torch.Size([64])

In [14]:
#%%time 
#torch.manual_seed(0) # SET SEED FOR TESTING
E = torch.randn(char_embed_sz, len(ctoi),     device=device, dtype=torch.float64, requires_grad=True) # embedding
Whz = torch.eye(nhidden,       nhidden,       device=device, dtype=torch.float64, requires_grad=True)
Whr = torch.eye(nhidden,       nhidden,       device=device, dtype=torch.float64, requires_grad=True)
Whh_ = torch.eye(nhidden,       nhidden,       device=device, dtype=torch.float64, requires_grad=True)
Uxh_ = torch.randn(nhidden,    char_embed_sz, device=device, dtype=torch.float64, requires_grad=True) # input converter
Uxz = torch.randn(nhidden,     char_embed_sz, device=device, dtype=torch.float64, requires_grad=True) # input converter
Uxr = torch.randn(nhidden,     char_embed_sz, device=device, dtype=torch.float64, requires_grad=True) # input converter
V = torch.randn(nclasses,      nhidden,       device=device, dtype=torch.float64, requires_grad=True) # take RNN output (h) and predict target

bx = torch.zeros(nhidden,      1,             device=device, dtype=torch.float64, requires_grad=True)
by = torch.zeros(nclasses,     1,             device=device, dtype=torch.float64, requires_grad=True)

# if using relu, b must be 0. W must be identity so don't mess with sd. others must have low stdev
# From [Le 2015] https://arxiv.org/abs/1504.00941
# "For IRNNs, in addition to the recurrent weights being initialized at identity, the non-recurrent
#  weights are initialized with a random matrix, whose entries are sampled from a
#  Gaussian distribution with mean of zero and standard deviation of 0.001."
sd = 0.001  # weight stddev init for relu
sd = 0.01   # weight stddev init for tanh
# with torch.no_grad():
#     E *= sd
#     U *= sd
#     V *= sd
    
# gradient clipping values 
gc = {1, 10, 100, 1000}

parameters = [E,Whz,Whr,Whh_,Uxz,Uxr,Uxh_,V]
optimizer = torch.optim.Adam(parameters, lr=0.0005, weight_decay=0.0)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=1)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, 
                                              mode='triangular2',
                                              step_size_up=3,
                                              base_lr=0.0005, max_lr=0.002,
                                              cycle_momentum=False)

history = []
epochs = 20
for epoch in range(1, epochs+1):
    H = torch.zeros(nhidden, nchunks, device=device, dtype=torch.float64, requires_grad=False)
    epoch_training_loss = 0.0
    epoch_training_accur = 0.0
    loss = 0
    for t in range(chunk_size-1):  # char t in chunk predicts t+1 so one less
        chars_step_t = X[:,t] # char_embed_sz x nchunks
        # column E[i] is the embedding for char index i. same as multiple E.mm(onehot(i))
        embedding_step_t = E[:,chars_step_t] # char_embed_sz x nchunks
#         print(embedding_step_t.shape, E.shape, H.shape, W.shape, U.shape)

        Z = torch.sigmoid(Whz@H + Uxz@embedding_step_t)
        R = torch.sigmoid(Whr@H + Uxr@embedding_step_t)
        H_ = torch.tanh(Whh_.mm(R*H) + Uxh_.mm(embedding_step_t))
        H = torch.tanh( (1-Z)*H + Z*H_ )

        o = V.mm(H)
        o = o.T # make it nchunks x nclasses
        p = softmax(o)
        correct = torch.argmax(p, dim=1)==y[:,t]
        epoch_training_accur += torch.sum(correct)
#         print(f"loss {loss:7.4f}")
        loss += F.cross_entropy(o, y[:,t])
        
        if t % bptt == 0 and t > 0:
#             print(f"gradient at {t:4d}, loss {loss.item():7.4f}")
            optimizer.zero_grad()
            loss.backward() # autograd computes U.grad, M.grad, ...
            optimizer.step()
            epoch_training_loss += loss.detach().item()
            loss = 0
            H = H.detach() # no longer consider previous computations

    epoch_training_accur /=  nchunks * (chunk_size-1)
    epoch_training_loss /= bptt * nchunks
    scheduler.step()
    
    print(f"Epoch {epoch:3d} training loss {epoch_training_loss:8.3f}   accur {epoch_training_accur:7.4f}   LR {scheduler.get_last_lr()[0]:7.6f}")

Epoch   1 training loss   17.855   accur  0.3731   LR 0.001000
Epoch   2 training loss    5.514   accur  0.5183   LR 0.001500
Epoch   3 training loss    4.552   accur  0.5525   LR 0.002000
Epoch   4 training loss    4.210   accur  0.5682   LR 0.001500
Epoch   5 training loss    3.661   accur  0.5953   LR 0.001000
Epoch   6 training loss    3.289   accur  0.6175   LR 0.000500
Epoch   7 training loss    2.997   accur  0.6375   LR 0.000750
Epoch   8 training loss    3.036   accur  0.6337   LR 0.001000
Epoch   9 training loss    3.095   accur  0.6288   LR 0.001250
Epoch  10 training loss    3.140   accur  0.6259   LR 0.001000
Epoch  11 training loss    2.981   accur  0.6381   LR 0.000750
Epoch  12 training loss    2.829   accur  0.6493   LR 0.000500
Epoch  13 training loss    2.684   accur  0.6612   LR 0.000625
Epoch  14 training loss    2.697   accur  0.6597   LR 0.000750
Epoch  15 training loss    2.720   accur  0.6579   LR 0.000875
Epoch  16 training loss    2.741   accur  0.6570   LR 0

In [15]:
def sample(initial_chars, n, temperature=0.1):
    "Derived from Karpathy: https://gist.github.com/karpathy/d4dee566867f8291f086"
    chars = initial_chars
    n -= len(initial_chars)
    with torch.no_grad():
        for i in range(n):
            h = torch.zeros(nhidden, 1, dtype=torch.float64, device=device, requires_grad=False)  # reset hidden state at start of record
            for t in range(len(chars)):  # for each char in a name
                c = chars[t]
                ci = ctoi[c]
                embedding_step_t = E[:,ci].reshape(char_embed_sz,1) # col is embedding for c; must be column
#                 print(embedding_step_t.shape, E.shape, h.shape, Whz.shape, Uxz.shape)#, V.shape)
                z = torch.sigmoid(Whz@h + Uxz@embedding_step_t)
                r = torch.sigmoid(Whr@h + Uxr@embedding_step_t)
                h_ = torch.tanh(Whh_.mm(r*h) + Uxh_.mm(embedding_step_t))
                h = torch.tanh( (1-z)*h + z*h_ )
            o = V@h
#             print(o.shape, h.shape)
            o = o.reshape(nclasses)
            p = softmax(o)
#             wi = torch.argmax(p) # this doesn't work (just repeats a million times)
            wi = np.random.choice(range(len(vocab)), p=p.cpu()) # don't always pick most likely; pick per distribution
            chars.append(vocab[wi])
    return chars

In [19]:
''.join( sample(list('yes we can'), 300) ) 

"yes we can't difference in government, now and when we have cour power of our country that means circulalbure, the marshall planes on this is our markers, not our country is in when a needs in a transition cooperation, the legal immediate economic u;lted with birthday part in populations back in you"